# Exercise 23 - Reduced Order Models with Finite Elements

### Task
Implement a reduced order model within a dynamic finite element simulation. This occurs in the `solve` member function of the `Dynamic_cantileverbeam`. Take a look at the two if statements `if phi is not None:`-

### Learning goals
- Understand how to identify a reduced basis from simulation data
- Understand how to implement a reduced basis within a dynamic finite element code
- Experience the speed-up accomplished by a reduced order model

In [ ]:
import numpy as np
import time
from scipy.sparse.linalg import spsolve
import warnings
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Identifier of reduced order basis

$\boldsymbol{\psi}$ which can be used as 
$$\boldsymbol{u}(t)\approx \boldsymbol{\psi} \boldsymbol{c}(t)$$
$$\boldsymbol{\psi}(t)^\intercal \boldsymbol{M}\boldsymbol{\psi}\ddot{\boldsymbol{c}}(t)+\boldsymbol{\psi}^\intercal \boldsymbol{K}\boldsymbol{\psi}\boldsymbol{c}(t)=\boldsymbol{\psi}^\intercal\boldsymbol{F}(t)$$

In [ ]:
def ROM(U, r):
    u, s, vh = np.linalg.svd(np.transpose(U))
    return u[:, :r], s

## Finite element method

**boundary condition helper**

In [ ]:
def applyDirichletBC(index, K):
    K[index, :] = 0.
    K[:, index] = 0.
    K[index, index] = 1.
    return K

**class used for basic 2D finite element formulation** (using linear quadrilatereal elements applied to a cantilever beam)

The class consists of the following member functions
- `__init__` as constructor initializing the global mass and stiffness matrix (and computing the local mass and stiffness matrix, which is the same for every element)
- `LocalStiffnessMatrix` computes the local stiffness matrix
- `LocalMassMatrix` computes the local mass matrix
- `GlobalSystem` assembles the global system consting of global mass and stiffness matrix and global force vector
- `ApplyDirichletBC` applies the Dirichlet boundary conditions to the global system
- `ApplyNeumannBC` applies the Neumann boundary conditions to the global system
- `Solve` solves the system (if a reduced basis `phi` is used, the **projection to the reduced basis** is performed within this member function)

In [ ]:
class Dynamic_cantileverbeam:
    def __init__(self, nx, ny, E, nu, rho, f, omega):
        self.nx = nx
        self.ny = ny
        x = np.linspace(0, 2 * nx, nx + 1)
        y = np.linspace(0, 2 * ny, ny + 1)
        self.X, self.Y = np.meshgrid(x, y)
        self.f = f
        self.omega = omega

        self.Kl = self.LocalStiffnessMatrix(E, nu)
        self.Ml = self.LocalMassMatrix(rho)
        self.eft = lambda i, j: np.array(
            [(i + 1) * 2 * (nx + 1) + 2 * j, (i + 1) * 2 * (nx + 1) + 2 * j + 1,  # element freedom table
             (i + 1) * 2 * (nx + 1) + 2 * j + 2, (i + 1) * 2 * (nx + 1) + 2 * j + 3,
             i * 2 * (nx + 1) + 2 * j + 2, i * 2 * (nx + 1) + 2 * j + 3,
             i * 2 * (nx + 1) + 2 * j, i * 2 * (nx + 1) + 2 * j + 1])
        # initial conditions
        self.u0 = np.zeros(2 * (nx + 1) * (ny + 1))
        self.du0 = np.zeros(2 * (nx + 1) * (ny + 1))

        self.K, self.M, self.F = self.GlobalSystem(nx, ny, self.Kl, self.Ml)
        self.K = self.ApplyDirichletBC(nx, ny, self.K)

    def LocalStiffnessMatrix(self, E, nu):
        Kl = np.zeros((8, 8))
        gp = np.sqrt(1. / 3.) * np.array([-1, 1])  # gauss points
        C = E / (1 - nu ** 2) * np.array([[1, nu, 0],  # plane stress
                                          [nu, 1, 0],
                                          [0, 0, 1 - nu]])
        for i in range(2):
            for j in range(2):
                xi = gp[i]
                eta = gp[j]
                dN = 0.25 * np.array([[-(1 - eta), -(1 - xi)],  # shape function derivatives
                                      [(1 - eta), -(1 + xi)],
                                      [(1 + eta), (1 + xi)],
                                      [-(1 + eta), (1 - xi)]])
                B = np.zeros((3, 8))
                for k in range(4):
                    B[:, 2 * k:2 * k + 2] = np.array([[dN[k, 0], 0],
                                                      [0, dN[k, 1]],
                                                      [dN[k, 1], dN[k, 0]]])
                Kl += np.dot(np.dot(np.transpose(B), C), B)
        return Kl

    def LocalMassMatrix(self, rho):
        Ml = np.zeros((8, 8))

        gp = np.sqrt(1. / 3.) * np.array([-1, 1])  # gauss points
        gw = np.array([1, 1])  # gauss weights

        for i in range(len(gp)):
            for j in range(len(gp)):
                xi = gp[i]
                eta = gp[j]
                N = 0.25 * np.array([[(1 - xi) * (1 - eta), 0, (1 + xi) * (1 - eta), 0, (1 + xi) * (1 + eta), 0,
                                      (1 - xi) * (1 + eta), 0],
                                     [0, (1 - xi) * (1 - eta), 0, (1 + xi) * (1 - eta), 0, (1 + xi) * (1 + eta), 0,
                                      (1 - xi) * (1 + eta)]])

                Ml += gw[i] * gw[j] * rho * np.dot(np.transpose(N), N)
        return Ml

    def GlobalSystem(self, nx, ny, Kl, Ml):
        ndofs = 2 * (nx + 1) * (ny + 1)  # number of degrees of freedom
        K = np.zeros((ndofs, ndofs))
        M = np.zeros((ndofs, ndofs))
        F = np.zeros((ndofs))

        for i in range(nx):
            for j in range(ny):
                eftij = self.eft(j, i)
                for k in range(8):
                    K[eftij, eftij[k]] += Kl[:, k]
                    M[eftij, eftij[k]] += Ml[:, k]
        return K, M, F

    def ApplyDirichletBC(self, nx, ny, K):
        # Dirichlet
        for i in range(ny + 1):  # x
            K = applyDirichletBC(self.eft(i, 0)[6], K)
        K = applyDirichletBC(self.eft(0, 0)[7], K)  # y
        return K

    def ApplyNeumannBC(self, nx, ny, f, omega, t, F):
        # Neumann    
        for i in range(ny + 1):  # y
            if i == 0 or i == ny:
                F[self.eft(i, nx - 1)[5]] = f * np.cos(omega * t) * 0.5 / ny
            else:
                F[self.eft(i, nx - 1)[5]] = f * np.cos(omega * t) / ny
        return F

    def Solve(self, n, dt, phi=None):

        if phi is not None:
            U = np.zeros((n + 2, phi.shape[1]))
            u0 = np.dot(self.u0, phi)
            u1 = np.dot(self.du0 * dt + self.u0, phi)
            F = np.dot(self.F, phi)
            K = np.dot(np.dot(np.transpose(phi), self.K), phi)
            M = np.dot(np.dot(np.transpose(phi), self.M), phi)
            U[0] = u0
            U[1] = u1
        else:
            U = np.zeros((n + 2,) + self.u0.shape)
            M = self.M
            K = self.K
            F = self.F
            u0 = self.u0
            u1 = self.du0 * dt + self.u0
            U[0] = u0
            U[1] = u1

        t = 0
        for i in range(n):
            F = self.ApplyNeumannBC(self.nx, self.ny, self.f, self.omega, t, self.F)
            if phi is not None:
                # your code goes here
            rhs = F - np.dot(K - 2. / dt ** 2 * M, u1) - np.dot(1. / dt ** 2 * M, u0)
            lhs = (1. / dt ** 2 * M)
            warnings.simplefilter("ignore")
            u2 = spsolve(lhs, rhs)
            u0 = u1
            u1 = u2
            t += dt
            U[i + 2] = u2
        if phi is not None:
            # your code goes here
        self.U = U
        return np.reshape(U[:, ::2], (n + 2, self.ny + 1, self.nx + 1)), np.reshape(U[:, 1::2],
                                                                                    (n + 2, self.ny + 1, self.nx + 1))

## Post-processing helpers

**field visualization with contourplot**

In [ ]:
def plot_field(x, y, z, title):
    # Set up plot
    fig, ax = plt.subplots()

    # Plot the field
    cp = ax.contourf(x, y, z, levels=12, cmap=plt.cm.jet)
    ax.pcolormesh(0.5 * (x[:-1, :-1] + x[1:, 1:]),
                  0.5 * (y[:-1, :-1] + y[1:, 1:]),
                  0.5 * (z[:-1, :-1] + z[1:, 1:]),
                  facecolor='None', edgecolors='k', linewidth=1)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Add a colorbar to the plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("top", size="25%", pad="20%")
    fig.colorbar(cp, cax=cax, format='%.2f', orientation="horizontal")
    cax.xaxis.set_ticks_position("top")

    ax.set_xlabel("$x$")
    ax.set_ylabel(title)

    fig.tight_layout()
    plt.show()

**field visualization including deformations**

In [ ]:
def plot_deformedfield(x, y, ux, uy, z, title, s=1):
    plot_field(x + ux * s, y + uy * s, z, title)

**visualization of time history at a selected node**

In [ ]:
def plot_timehistory_at_node(x, y, z, x0, y0, dt, ylabel):
    n = len(z)
    z = z[:, y0 // 2, x0 // 2]

    t = np.linspace(0, dt * (n), n)
    fig, ax = plt.subplots()

    plt.plot(t, z, 'k')

    ax.set_xlabel('$t$')
    ax.set_ylabel(ylabel)

    fig.tight_layout()
    plt.show()

**modeshape visualization**

In [ ]:
def plot_modeshapes(phi, x, j, s, dim=None, modeshape=None):
    ny, nx = x.shape
    ny = ny - 1
    nx = nx - 1
    phi[:, j][dim::2] = phi[:, j][dim::2] / max(abs(phi[:, j][dim::2])) * s
    fig, ax = plt.subplots()

    for i in range(ny + 1):
        plt.plot(x[0], phi[:, j][dim::2][i * (nx + 1):(i + 1) * (nx + 1)] + 2 * i, 'o', color='k')
    if modeshape is not None:
        plt.plot(x[0], modeshape(x[0]) * s + ny, color='r', label='Analytic Mode')
        # ax.legend()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.minorticks_off()
    fig.tight_layout()
    plt.show()

**plotting of modeshape coefficients**

In [ ]:
def plot_modeshape_coefficients(s, r):
    fig, ax = plt.subplots()
    ax.scatter(np.linspace(1, len(s[:5 * r]), len(s[:5 * r])), s[:5 * r], color='k', marker='o')
    ax.set_yscale('log')
    plt.minorticks_off()
    ax.set_xlabel('index')
    ax.set_ylabel('coefficient')
    fig.tight_layout()
    plt.show()

## Pre-processing

**mesh parameters**

In [ ]:
# your code goes here: define the mesh parameters nx and ny

**physical parameters**

In [ ]:
# your code goes here: define the parameters E, nu, rho, f, omega = 0

**simulation for 5000 timesteps as snapshot collection**

In [ ]:
model = Dynamic_cantileverbeam(nx, ny, E, nu, rho, f, omega)
n = 5000
dt = 5e-6
start = time.perf_counter()
Ux, Uy = model.Solve(n, dt)
end = time.perf_counter()
print("Elapsed time: {:.2f}".format(end - start))

**visualization of four snapshots**

In [ ]:
for i in range(4):
    plot_deformedfield(model.X, model.Y, Ux[(i + 1) * 1000], Uy[(i + 1) * 1000], Ux[(i + 1) * 1000],
                       "$t={:.1e}$".format((i + 1) * 1000 * dt) + ",  $u_{1}$", s=2)

**time history at the central node at the right edge**

In [ ]:
plot_timehistory_at_node(model.X, model.Y, Uy, nx * 2, ny, dt, "$u_{2}$")

## Reduced order model

**truncation level**

In [ ]:
r = 5

**number of timesteps with reduced order model**

In [ ]:
nr = 10 * n

**identification of reduced basis**

In [ ]:
phi, s = ROM(model.U, r)

**simulation with reduced basis**

In [ ]:
start = time.perf_counter()
Uxr, Uyr = model.Solve(nr, dt, phi)
end = time.perf_counter()
print("Elapsed time: {:.2f}".format(end - start))

**Reduction in numbers of degrees of freedom**

In [ ]:
print("full system: {:d}\t reduced system: {:d}".format(2 * (nx + 1) * (ny + 1), r))

**time history at central node at right edge**

In [ ]:
plot_timehistory_at_node(model.X, model.Y, Uyr, nx * 2, ny, dt, "$\\hat{u}_{2}$")

## Modeshape analysis

**analytical mode shapes of a Euler-Bernoulli beam**

In [ ]:
modeshape = lambda x, beta, sigma: (np.cosh(beta / (2 * nx) * x) - np.cos(beta / (2 * nx) * x)
                                    - sigma * (np.sinh(beta / (2 * nx) * x) - np.sin(beta / (2 * nx) * x)))
modeshape_norm = lambda x, beta, sigma: modeshape(x, beta, sigma) / max(abs(modeshape(x, beta, sigma)))

**comparison with the identified modeshapes**

In [ ]:
plot_modeshapes(phi, model.X, 0, 10, 1, lambda x: modeshape_norm(x, 1.875, 0.7341))
plot_modeshapes(phi, model.X, 1, 10, 1, lambda x: modeshape_norm(x, 4.694, 1.0185))
plot_modeshapes(phi, model.X, 2, 10, 1, lambda x: modeshape_norm(x, 7.855, 0.9992))
plot_modeshapes(-phi, model.X, 3, 10, 1, lambda x: modeshape_norm(x, 10.996, 1))

**modeshape coefficients**

In [ ]:
plot_modeshape_coefficients(s, r)